In [14]:
import librosa
import numpy as np
from scipy.signal import lfilter
import sys
sys.path.append('../../')
import utils as utils

import pdb

In [15]:
SR = 44100
fs = 8192 # sampling frequency in Hz
n_samples = round(fs) # Length of audio signal for individual clap 0.2 sec
rise_time = round(0.0032*fs) # Exponentions rising attack segment of envelope 3.2ms
base = np.power(0.99, (SR/fs)) # Make shape of envelope independent from fs
R = 0.9 # pole radius of cavity resonator
theta = np.pi/6 # resnonant frequency of cavity resonator: adjust for different hand clapping modes; higher for flat hands
b = np.array([0.2]) # cavity transfer function numerator
a = np.array([1, -2*R*np.cos(theta), R*R]) # cavity transfer function denominator
t = np.linspace(0, n_samples, n_samples) # create time vector

env = np.power(base, abs(rise_time-t)) #Envelope (same for attack/decay)

avgOOI = 0.4 # Natural 0.4 Enthusiastic 0.3 Bored 0.6
stdOOI = avgOOI/16;
out = []
for i in range(10):
    x=np.random.random(n_samples)*env
    #pdb.set_trace()
    y=lfilter(b,a,x)
    out = np.append(out, np.pad(y, (0, int((stdOOI*np.random.random()+avgOOI)*fs))))
    #out.append((stdOOI*np.random.random()+avgOOI)*fs);

#pdb.set_trace()
utils.audio.playSingleAudio(out, SR)

In [105]:
t

array([1.000e+00, 2.000e+00, 3.000e+00, ..., 8.190e+03, 8.191e+03,
       8.192e+03])

In [35]:
## Jake et al. Real-Time Sound Synthesis of Audience Applause

# This paper uses a different style of (single) hand clap generation
# TODO: add different hand types
fs=8192                            # Sampling frequency in Hz
n_samples=round(fs*10)              # Total length of audio clip generated in seconds
#NClap=round(Fs*0.1);               # Length of audio signal for individual clap
#RiseTime=round(0.032*Fs);          # Exponentially rising attack segment of envelope 3.2 ms
#Base=0.99^(44100/Fs);              # Make shape of envelope independent from Fs
#BaseDecay=Base^0.1;                # Slower decay of envelope to fake reverb
#R=0.9;                             # Pole radius of cavity resonator
theta=np.pi/2.5                       # Resonant frequency of cavity resonator: adjust for different hand clapping styles; higher for flat hands 
theta_std=np.pi/6                      # Vary hand clapping styles
#b=1;                               # Cavity transfer function numerator - arbitary
#env=min(Base.^(RiseTime-t),BaseDecay.^(t-RiseTime)); % Generate envelope - fast rise, slow decay
avgOOI=0.4                     # Time between clap in secs. Natural 0.4 Enthusiastic 0.3 Bored 0.6
stdOOI=avgOOI/16             # Time between claps varies with this st dev
swell=1                            # Applause increases over Swell seconds at the beginning
fade=5                             # Applause fades over Fade seconds at the end
out_left=np.zeros(n_samples)          # Left channel
out_right=np.zeros_like(out_left)                   # Right Channel

#tri = makedist('Triangular','a',0.15,'b',0.22,'c',0.29); % Create triangular distribution between for 150ms < OOI < 290ms

#LeadOOI = 0.44;                    # Lead Onset-to-Onset interval; ms 190?500 range; For Synch Only
attack_time=0.032                   # Attack Time in ms; TODO: randomize length, add into loop with randn
attack_time_n=round(attack_time*fs)  # Exponentially rising attack segment
decay_time=0.052                    # Decay Time in ms; TODO: randomize length, add into loop with randn
decay_time_n=round(decay_time*fs)    # Exponentially decay attack segment

#NClap=attackTime_n+decayTime_n    # Length of Clap; TODO: variable length

n_claps=fs*1                         # Length of 1 Clap, here 1 sec
#NClap=floor(sum(duration)*Fs);
t = np.linspace(1, n_claps, n_claps) # create time vector

max_gain=0.5                        # Max. Gain
c_attack=44100/fs                  # Constant for attack type; fast vs. slow rise
c_decay=44100/fs*0.1               # Constant for decay type; smaller(+longer) decay -> higher reverberation/echo

#pdb.set_trace()

env=np.min(np.stack([np.power(0.99, (c_attack*(attack_time_n-t))), np.power(0.99, (c_decay*(t-attack_time_n)))]), axis=0)

#plt.plot(env)

import matplotlib.pyplot as plt
#plt.plot(env)
#pdb.set_trace()
row_n=50                        # Number of rows
pp_per_row=1                    # People per row
n_person=200                       # Number of persons clapping -------------------------------------
if (row_n*pp_per_row)<n_person:       # Check if enough seats are available
    print("More people than seats available")

row=0

# Impulse response of different locations
# UNCOMMENT if you want to use impulse response
#[imp, Fs_imp] = audioread('impulse_responses/impulse_room.wav');
#[imp, Fs_imp] = audioread('impulse_responses/impulse_bathroom.wav');
#[imp, Fs_imp] = audioread('impulse_responses/impulse_cathedral.wav');
for i in range(1, n_person):
    #a=[1 -2*R*cos(Theta+(rand-0.5)*ThetaStD) R*R];  # Cavity transfer function denominator
#   Onset=round(rand*AverageOOI*Fs)+1;              # first clap onset uniformly distributed without clapping interval
    onset=round(np.random.random()*fs*swell)+1                   # first clap onset uniformly between 0 and Swell seconds
    end_clap=n_samples-np.random.random()*fs*fade                  # clapping stops, uniformly distributed over Fade seconds at the end
    alpha=np.random.random()                                     # random distribute between left and right channel
    
    if np.mod(i-1, pp_per_row):                         # Fill rows; increment index when row is full 10 persons / row
        row = row + 1

    beta=np.power(i,-0.5)
    #beta = max_gain-0.2*row/row_n  
    
    OOI = (avgOOI+np.random.random()*stdOOI)              # Random OOI, no (a)synch 
    
    while(onset+n_claps<end_clap):
        x=np.random.random(n_claps)*env                      # Generate single clap
        #y=filter(b,a,x);                           # Filter single clap
        #pdb.set_trace()

        # Filter single clap
        y_bp1 = bandpass_(x, 10, 2000, fs)
        #plt.plot(y_bp1)
        #pdb.set_trace()
        y_hp1 = highpass_(y_bp1, 10, 500, fs)
        y_lp1 = lowpass_(y_hp1, 10, 2500, fs)
        
        y_bp2 = bandpass_(x, 10, 1000, fs)
        y_hp2 = highpass_(y_bp2, 10, 500, fs)
        y_lp2 = lowpass_(y_hp2, 10, 2500, fs)
        y=(y_lp1 - y_lp2)
        
        
        
        #reverb = reverberator('PreDelay', 0.5, 'WetDryMix', 1, 'SampleRate', Fs);
        #y = reverb(y);
        #pdb.set_trace()
        out_left[onset:onset+n_claps]=out_left[onset:onset+n_claps]+beta*alpha*y
        out_right[onset:onset+n_claps]=out_right[onset:onset+n_claps]+beta*(1-alpha)*y

        onset=onset+round(fs*OOI)


# UNCOMMENT NEXT 3 LINES if you want to use impulse response
#imp = resample(imp, Fs, Fs_imp);
#outleft = fconv(outleft, imp);
#outright = fconv(outright, imp);
out_stereo=np.stack([out_left, out_right])
print(out_stereo.mean(0).shape)
utils.audio.playSingleAudio(out_stereo, fs)

More people than seats available
(81920,)


In [4]:
## Functions

#lowpass(1+np.cos(w0)/2) *
def lowpass_(x, q_db, f0, Fs):
    # q_db: quality factor
    # f_0: center frequency
    # Fs: sampling frequency
    w0 = 2*np.pi*f0/Fs
    alpha_q_db = np.sin(w0)/(2*np.power(10,(q_db/20)))
    
    b = (1-np.cos(w0)/2) * np.array([1, 2, 1])
    a = np.array([1+alpha_q_db, -2*np.cos(w0), 1-alpha_q_db])
    return lfilter(b, a, x)

# highpass
def highpass_(x, q_db, f0, Fs):
    # q_db: quality factor
    # f_0: center frequency
    # Fs: sampling frequency
    
    w0 = 2*np.pi*f0/Fs
    alpha_q_db = np.sin(w0)/(2*np.power(10,(q_db/20)))
    
    b = (1+np.cos(w0)/2) * np.array([1, -2, 1])
    a = np.array([1+alpha_q_db, -2*np.cos(w0), 1-alpha_q_db])
    return lfilter(b, a, x)

#bandpass
def bandpass_(x, q_db, f0, Fs):
    # q_db: quality factor
    # f_0: center frequency
    # Fs: sampling frequency
    #pdb.set_trace()
    w0 = 2*np.pi*f0/Fs
    alpha_q_db = np.sin(w0)/(2*np.power(10, (q_db/20)))
    
    b = np.array([1, 0, -1])
    a = np.array([1+alpha_q_db, -2*np.cos(w0), 1-alpha_q_db])
    return lfilter(b, a, x)

"""
Find 2^n that is equal to or greater than.
"""
def nextpow2(i):
    n = 1
    while n < i: n *= 2
    return n

#   This is internal function used by fft(), because the FFT routine
#   requires that the data size be a power of 2.

def fconv(x, h):
    # FCONV Fast Convolution
    # [y] = FCONV(x, h) convolves x and h, and normalizes the output
    # to +-1.
    # x = input vector
    # h = input vector
    #
    Ly=len(x)+len(h)-1 #
    if nextpow2(Ly)==0: print("nextpow2")
    Ly2=np.power(2, nextpow2(Ly))         # Find smallest power of 2 that is > Ly
    X=np.fft.fft(x, n=Ly2)                  # Fast Fourier transform
    H=np.fft.fft(h, n=Ly2)                  # Fast Fourier transform
    Y=X*H                         # DO CONVOLUTION
    y=np.fft.irfft(Y, n=Ly2)           # Inverse fast Fourier transform
    y=y[1:1:Ly]                    # Take just the first N elements
    return y/max(abs(y))                # Normalize the output
